In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /content/drive/MyDrive/UCD/ECS-289G-DL
import sys
sys.path.append('/content/drive/MyDrive/UCD/ECS-289G-DL')

In [ ]:
!pip install pytorch_lightning
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets

In [ ]:
from auto_encoder import AutoEncoder
from masked_dataset import CIFAR10DataModule, IMDBDataModule
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
from pytorch_lightning.loggers import TensorBoardLogger
from datetime import timedelta


In [ ]:
imdb = IMDBDataModule()

In [ ]:
in_channel_size = 3
num_epochs = 10
model = AutoEncoder(in_channel_size=262)

trainer = pl.Trainer(
    default_root_dir=os.getcwd(),
    logger=TensorBoardLogger(save_dir=os.getcwd(), version=2, name="lightning_logs"),
    auto_lr_find=True, 
    auto_scale_batch_size=True,
    accelerator="auto",
    log_every_n_steps=1, 
    max_epochs=50, 
    min_epochs=3,
    callbacks=[
        ModelCheckpoint(save_last=True, train_time_interval=timedelta(minutes=30)),
        LearningRateMonitor("epoch"),
        EarlyStopping(monitor="val_loss", mode="min", patience=3)
    ])
trainer.logger._log_graph = True

In [ ]:
trainer.tune(model, imdb)

In [ ]:
trainer.fit(model, imdb)